In [ ]:
# NOTE: pip shows imcompatible errors due to preinstalled libraries but you do not need to care
!pip install -q espnet==0.10.6 pyopenjtalk==0.2 pypinyin==0.44.0 parallel_wavegan==0.5.4 gdown==4.4.0 espnet_model_zoo pydub

In [ ]:
#@title English multi-speaker pretrained model { run: "auto" }
lang = 'English'
tag2 = 'kan-bayashi/libritts_xvector_vits' #@param /["kan-bayashi/vctk_gst_tacotron2", "kan-bayashi/vctk_gst_transformer", "kan-bayashi/vctk_xvector_tacotron2", "kan-bayashi/vctk_xvector_transformer", "kan-bayashi/vctk_xvector_conformer_fastspeech2", "kan-bayashi/vctk_gst+xvector_tacotron2", "kan-bayashi/vctk_gst+xvector_transformer", "kan-bayashi/vctk_gst+xvector_conformer_fastspeech2", "kan-bayashi/vctk_multi_spk_vits", "kan-bayashi/vctk_full_band_multi_spk_vits", "kan-bayashi/libritts_xvector_transformer", "kan-bayashi/libritts_xvector_conformer_fastspeech2", "kan-bayashi/libritts_gst+xvector_transformer", "kan-bayashi/libritts_gst+xvector_conformer_fastspeech2", "kan-bayashi/libritts_xvector_vits"] {type:"string"}
vocoder_tag2 = "none" #@param ["none", "parallel_wavegan/vctk_parallel_wavegan.v1.long", "parallel_wavegan/vctk_multi_band_melgan.v2", "parallel_wavegan/vctk_style_melgan.v1", "parallel_wavegan/vctk_hifigan.v1", "parallel_wavegan/libritts_parallel_wavegan.v1.long", "parallel_wavegan/libritts_multi_band_melgan.v2", "parallel_wavegan/libritts_hifigan.v1", "parallel_wavegan/libritts_style_melgan.v1"] {type:"string"}

In [ ]:
import time
from espnet2.bin.tts_inference import Text2Speech
from espnet2.utils.types import str_or_none
import torch
from IPython.display import display, Audio
import glob
import os
import numpy as np
import kaldiio
import pydub
from pydub import AudioSegment
from pydub.playback import play
import soundfile as sf
from espnet_model_zoo.downloader import ModelDownloader

In [ ]:

tag2 = 'kan-bayashi/libritts_xvector_vits'
vocoder_tag2 = "none"


text2speech = Text2Speech.from_pretrained(
    model_tag=str_or_none(tag2),
    vocoder_tag=str_or_none(vocoder_tag2),
    #device="cuda",
    speed_control_alpha=0.95,
    noise_scale=0.333,
    noise_scale_dur=0.3,
)

d = ModelDownloader()
model_dir = os.path.dirname(d.download_and_unpack(tag2)["train_config"])

# X-vector selection
spembs = None
if text2speech.use_spembs:
    xvector_ark = [p for p in glob.glob(f"{model_dir}/../../dump/**/spk_xvector.ark", recursive=True) if "tr" in p][0]
    xvectors = {k: v for k, v in kaldiio.load_ark(xvector_ark)}
    spks = list(xvectors.keys())

    # randomly select speaker
    random_spk_idx = np.random.randint(0, len(spks))
    spk = spks[random_spk_idx]
    spembs = xvectors[spk]#xvectors[spks[2]]
    print(f"selected spk: {spk}")

# Speaker ID selection
sids = None #'p300'
if text2speech.use_sids:
    spk2sid = glob.glob(f"{model_dir}/../../dump/**/spk2sid", recursive=True)[0]
    with open(spk2sid) as f:
        lines = [line.strip() for line in f.readlines()]
    sid2spk = {int(line.split()[1]): line.split()[0] for line in lines}
    
    # randomly select speaker
    sids = np.array(np.random.randint(1, len(sid2spk)))
    spk = sid2spk[int(sids)]
    print(f"selected spk1: {spk}")

# Reference speech selection for GST
speech = None
if text2speech.use_speech:
    # you can change here to load your own reference speech
    # e.g.
    # import soundfile as sf
    # speech, fs = sf.read("/path/to/reference.wav")
    # speech = torch.from_numpy(speech).float()
    speech = torch.randn(50000,) * 0.01


In [ ]:
tts="""This section helps you categorize your problem in order to realize the role of AI/ML in finding a solution to your business problem. 
Which bucket does your problem fall under? 
A First Principles problem can be realized when an existing approach to address an issue can be found.  In short, there exists a known methodology to solve the problem. Methodical disassembling of a computer to repair its defective parts may be considered as a first principles problem. 
The emotional, strategic, legal, and ethical bucket of problems are typically found in your business. Picking an ideal applicant for employment, addressing a customer churn problem, avoiding discrimination and bias, conducting business within your personal & professional morals and values some business scenarios that fall under this bucket.
A logical or a rational business problem is that which you know has reason but not know of an approach towards its solution.  
Ensure to categorize your business problem into one of these buckets, divide the problem further, if necessary. This helps to optimize your AI/ML approach to its best potential.  
If your organization is in the beginning phases of adapting AI, we recommend for you to pick problems that fall under the third bucket.
Why? 
For solving the first bucket, AI/ML mostly aid in minimizing the number of choices or parameters required by a human expert to execute the known optimal approach. The second bucket requires multiple human experts and AI/ML systems to find an optimal solution. However, AI/ML systems are just aids for the experts.
The third bucket is where an AI/ML system is your front-end problem-solving contributor. Predicting a stock price can be a good example for this bucket. Picking third-bucket problems will thus help you immediately realize the value of AI/ML systems.  
If there is insufficient data for you to incorporate the systems, focus on collecting the data.  
For any third-bucket problem, it is efficient to quickly incorporate AI/ML systems for problem-solving."""

'2428_83705'**,** '251_137823'**,** '251_118436'**,** '3752_4944'**,** '422_122949'

tag2 = 'kan-bayashi/libritts_xvector_vits'

vocoder_tag2 = "none"

In [ ]:
y=['251_137823','251_118436','422_122949']
#1:251_137823,2:251_118436,3:422_122949

In [ ]:
j=1
for i in y:
  with torch.no_grad():
    start = time.time()
    wav = text2speech(tts, speech=speech, spembs=xvectors[i], sids=sids)["wav"]
    rtf = (time.time() - start) / (len(wav) / text2speech.fs)
    print(f"RTF = {rtf:5f}")
    print(i)
    display(Audio(wav.view(-1).cpu().numpy(),rate=text2speech.fs))
    sf.write("speech_"+str(j)+".wav", wav.to('cpu').numpy(), 22050)
    sound = pydub.AudioSegment.from_wav("/content/speech_"+str(j)+".wav")
    sound.export("voice_"+str(j)+".mp3", format="mp3")
    j=j+1